In [ ]:
!nvidia-smi

In [ ]:
import json
token = {"username":"","key":""}#使用你自己的用户和密钥 Use your own username and key
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

In [ ]:
!kaggle datasets download -d gpiosenka/cards-image-datasetclassification

In [ ]:
!unzip '/content/datasets/gpiosenka/cards-image-datasetclassification/cards-image-datasetclassification.zip'

In [ ]:
import os
from torch.utils.data import Dataset 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
import time
import torch as t 
from torchvision import transforms as T 
from PIL import Image  
from torch.optim import lr_scheduler

In [ ]:
class Cards(Dataset):
  def __init__(self, train = True, mode = 'train\n'):
      
    data_dir = '.'#相对路径就能运行，好家伙
    fname = os.path.join(data_dir, 'cards.csv')
    with open(fname, 'r') as f:
        lines = f.readlines()[1:] 
    self.imgs = []
    
    for i in lines:
        if i.split(',')[4] == mode:
            self.imgs.append(i.split(',')) 
    for line in self.imgs:
      if line[1] == 'train/ace of clubs/output':
          print('Pre handle the data')
          self.imgs.remove(line)
    normalize = T.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
    if not train:
        # 验证集或测试集
        self.transform = T.Compose([
            T.Resize(224),#最短边224像素
            T.CenterCrop(224),#中心裁剪
            T.ToTensor(),#转成tensor
            normalize#归一化
        ])
    else:
        # 训练集，做数据增广
        self.transform = T.Compose([
            T.Resize(224),
            T.RandomResizedCrop(224),
            T.RandomHorizontalFlip(),#按概率水平翻转
            T.ToTensor(),
            normalize
        ])

  def __getitem__(self, index):
    img_line = self.imgs[index]
    img_path = './'+img_line[1]
    label = int(img_line[0])
    data = Image.open(img_path)
    data = self.transform(data)
    return data, label

  def __len__(self):
    return len(self.imgs)

In [ ]:
class basic_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(basic_block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.bn1(y)
        y = self.conv2(y)
        y = self.bn2(y)
        return F.relu(x + y)

class basic_block2(nn.Module):
    def __init__(self,in_channels,out_channels):# 3-64 64-128 128-256
        super(basic_block2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=2,padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=2,padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels,out_channels,kernel_size=1,stride=1,padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels)
    def forward(self, x):
        z = self.conv1(x)
        z = self.bn1(z)
        y = F.relu(self.bn2(self.conv2(x)))
        y = self.conv3(y)
        y = self.bn3(y)
        return F.relu(y+z)


In [ ]:
class Resnet18(nn.Module):
    '''按照网络结构图直接连接，确定好通道数量就可以'''

    def __init__(self):
        super(Resnet18, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.res2 = basic_block(64, 64)
        self.res3 = basic_block(64, 64)
        self.res4 = basic_block2(64, 128)
        self.res5 = basic_block(128, 128)
        self.res6 = basic_block2(128, 256)
        self.res7 = basic_block(256, 256)
        self.res8 = basic_block2(256, 512)
        self.res9 = basic_block(512, 512)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(512, 53)

    def forward(self, x):
        x = self.conv1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.res4(x)
        x = self.res5(x)
        x = self.res6(x)
        x = self.res7(x)
        x = self.res8(x)
        x = self.res9(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

In [ ]:
def main():  # dataset_1 dataset-resized
    train_data = Cards(train=True,mode = 'train\n')
    test_data = Cards(train=False,mode = 'test\n')
    train_loader = DataLoader(
        train_data,
        batch_size=128,
        shuffle=True,
        num_workers=0
    )
    test_loader = DataLoader(
        test_data,
        batch_size=128,
        shuffle=False,
        num_workers=0
    )
    model = AlexNet()
    print(model)
    device = torch.device('cuda:0')
    model = model.to(device)
    criteon = nn.CrossEntropyLoss().to(device)#交叉熵损失
    optimizer = optim.SGD(model.parameters(), lr=0.01)#优化器
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[20], gamma=0.1)
    best_val_acc = 0
    for epoch in range(50):
        model.train()
        since = time.time()
        for x, label in train_loader:
            x, label = x.to(device), label.to(device)#传给GPU
            logits = model(x)#返回值
            loss = criteon(logits, label)#计算损失
            optimizer.zero_grad()#梯度清零
            loss.backward()
            optimizer.step()
        print('Epoch: ', epoch, '训练集 loss: ', loss.item())#item是转成数字
        model.eval()
        with torch.no_grad():
            # 测试集
            total_correct = 0
            total_num = 0
            for x, label in test_loader:
                x, label = x.to(device), label.to(device)
                logits = model(x)
                loss = criteon(logits, label)
                pred = logits.argmax(dim=1)#获得每行最大值列号
                correct = torch.eq(pred, label).float().sum().item()
                total_correct += correct
                total_num += x.size(0)
            val_acc = total_correct / total_num
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), "./model_parameter.pkl")
            time_elapsed = time.time() - since
            print('Epoch: ', epoch, '测试集 loss: ', loss.item())#item是转成数字
            print('Epoch: ',epoch,' Training complete in {:.0f}min {:.0f}seconds'.format(time_elapsed // 60, time_elapsed % 60))
            timeCost = 'Training time {:.0f}min {:.0f}seconds'.format(time_elapsed // 60, time_elapsed % 60)
            print('Epoch: ',epoch, ' test acc: ', val_acc)
        scheduler.step()
    print('The best acc is '+ str(best_val_acc))


In [ ]:
main()